In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reasoning Engine in Vertex AI - Prompt template
@forusone (shins777@gmail.com)

This colab explains how to deal with prompt template for better query and reasoning

References :  
*  https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.preview.reasoning_engines
*  https://cloud.google.com/vertex-ai/generative-ai/docs/reasoning-engine/develop#prompt-template
*  https://python.langchain.com/v0.2/docs/concepts/#prompt-templates



## Init Vertex AI SDK

In [15]:
!pip install --upgrade --quiet \
    "google-cloud-aiplatform[langchain,reasoningengine]" \
    "google-cloud-firestore" \
    "langchain-google-firestore" \
    cloudpickle==3.0.0 \
    pydantic==2.7.4 \
    requests

In [16]:
import sys
from IPython.display import display, Markdown

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Configuration and initialization

In [17]:
# Static configuration
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://reasoning_7424"
MODEL = "gemini-1.5-flash"

# Firestore
DATABASE="chat-history"
COLLECTION="history"

### Initialize Vertex AI

In [18]:
import vertexai
from vertexai.preview import reasoning_engines

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Firestore setting for chat history

### Helper method to get session history

In [25]:

# refer to : https://python.langchain.com/v0.2/docs/integrations/memory/google_firestore/#custom-client

def get_session_history(session_id: str):
    """
    Retrieve chat history data with session id for each conversation from Firestore on GCP
    """

    from langchain_google_firestore import FirestoreChatMessageHistory
    from google.cloud import firestore

    client = firestore.Client(project=PROJECT_ID, database=DATABASE)

    return FirestoreChatMessageHistory(
        client=client,
        session_id=session_id,
        collection=COLLECTION,
        encode_message=False,
    )

def get_converstion(chat_history):
  """
  Get conversation from chat history
  """

  from langchain.schema import HumanMessage, AIMessage
  import langchain_core.messages

  for message in chat_history.messages:
      if type(message) is HumanMessage:
        print(f"Human : {message.content}")
      elif type(message) is AIMessage:
        print(f"AI : {message.content}")

def interactive_chat(agent, query, session_id):
  """
  Chat with agent with chat history.
  """
  respone = agent.query(input=query,
                        config={"configurable": {"session_id": session_id}},
  )
  return respone['output']


### Unit test of Firestore

In [26]:
# Tool define
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
  """Retrieves the exchange rate between two currencies on a specified date.
     지정된 날짜의 두 통화 간 환율을 검색합니다.
  """

  import requests

  response = requests.get(
      f"https://api.frankfurter.app/{currency_date}",
      params={"from": currency_from, "to": currency_to},
  )
  return response.json()

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents.format_scratchpad.tools import format_to_tool_messages

custom_prompt_template = {
    "user_input": lambda x: x["input"],
    "history": lambda x: x["history"],
    "agent_scratchpad": lambda x: format_to_tool_messages(x["intermediate_steps"]),
} | ChatPromptTemplate.from_messages([
    ("placeholder", "{history}"),
    ("user", "{user_input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [29]:
from time import perf_counter

session_id = "conv-004"

agent = reasoning_engines.LangchainAgent(
    model=MODEL,
    prompt=custom_prompt_template,
    chat_history=get_session_history,
    tools=[get_exchange_rate],
)

while True:
  query = input('사용자: ')

  if query == '종료': break

  t1_start = perf_counter()
  output = interactive_chat(agent, query, session_id)
  t1_end = perf_counter()

  display(Markdown(f"Elapse time : {t1_end-t1_start}"))
  display(Markdown(f"AI Agent : {output}"))
  print(f"------------------------------------ ")

chat_history = get_session_history(session_id)
chat_history.clear()
print(chat_history.messages)

사용자: 원달러 환율을 알려주세요.


Elapse time : 2.6321101720004663

AI Agent : 오늘 원달러 환율은 1,388.89원입니다. 


------------------------------------ 
사용자: 원달러 환율이 얼마인가요 ?


Elapse time : 2.0928335530006734

...

AI Agent : 오늘 원/달러 환율은 1,388.89원입니다. 




------------------------------------ 
사용자: 원 위안 환율은 ?


Elapse time : 1.243261433999578

AI Agent : 오늘 원/위안 환율은 100원당 0.52위안입니다. 




------------------------------------ 
사용자: 종료
[]
